<a href="https://colab.research.google.com/github/Vishweshwar133/Ecommerce/blob/main/Finetune_VITS_112922.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fine tune a VITS model with the Coqui TTS framework using audio samples of your choice.**

Thank you to all of the [Coqui TTS](https://https://github.com/coqui-ai/TTS) contributors, and [@Thorsten-Voice](https://https://www.youtube.com/c/ThorstenMueller ) (I think he wrote this original script). I have no idea who wrote the rnnoise loop, but they are great, too. I just smash things together. -nn

If you have run the script before, and want to examine your training session using Tensorboard, skip to the bottom section and run cells A-C, and then the Tensorboard cells.

1. Run this cell to connect your Google Drive account to save files.

In [1]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


2. Download and Build Rnnoise and Requirements (Optional if not processing .wav files for a new dataset. Multiple passes of rnnoise over samples may degrade audio quality. You should also preview your output dataset before training to ensure that the processing has not degraded any of your samples.  Occasionally, it will denoise too much of the tail end of phrases)

In [ ]:
!pip install pyloudnorm
!git clone https://github.com/xiph/rnnoise.git
!sudo apt-get install curl autoconf automake libtool python-dev pkg-config sox
%cd /content/rnnoise
!sh autogen.sh
!sh configure
!make clean
!make

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'rnnoise'...
remote: Enumerating objects: 420, done.
remote: Counting objects: 100% (219/219), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 420 (delta 194), reused 192 (delta 192), pack-reused 201
Receiving objects: 100% (420/420), 836.04 KiB | 7.40 MiB/s, done.
Resolving deltas: 100% (221/221), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'python-dev-is-python2' instead of 'python-dev'
autoconf is already the newest version (2.69-11.1).
autoconf set to manually installed.
automake is already the newest version (1:1.16.1-4ubuntu6).
automake set to manually installed.
pkg-config is already the newest version (0.29.1-0ubuntu4).
curl is already the newest version (7.68.0-1ubuntu2.18).
The following additional packages will be installed:
  libopencore-amrnb0 libopencore-amrwb0 libpyth

3. Install Coqui TTS, espeak-ng phonemeizer, download Coqui TTS source and examples from GitHub

In [ ]:
%cd /content
!sudo apt-get install espeak-ng
!git clone https://github.com/coqui-ai/TTS.git
!pip install TTS
!tts --list_models

/content
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,215 kB of archives.
After this operation, 12.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcaudio0 amd64 1.1-4 [7,908 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libsonic0 amd64 0.2.0-8 [13.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 espeak-ng-data amd64 1.50+dfsg-6 [3,682 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/main amd64 libespeak-ng1 amd64 1.50+dfsg-6 [189 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/universe amd64 espeak-ng amd64 1.50+dfsg-6 [322 kB]
Fetched 4,215 kB in 1s (3,483 kB/s)
deb

4. Download VITS model and Generate Sample Wav File to /content/ljspeech-vits.wav  This will be deleted when your Colab session is closed.

In [ ]:
!tts --text "I am the very model of a modern Major General" --model_name "tts_models/en/ljspeech/vits" --out_path /content/ljspeech-vits.wav

Process clips with rnnoise:
5. (Optional) Set path for original .wav files
Process Audio Clips using rnnoise.py - convert to 22khz mono.  Original author unknown, but thank you for this helpful code.

In [ ]:
#Location of original wav files. Directory must be named original.
orig_wavs="/content/drive/MyDrive/original"

6. (Optional) Run rnnoise, processed clips will be saved to your Google Drive in a folder named 'converted'

In [ ]:
from pathlib import Path

import os
import subprocess
import soundfile as sf
import pyloudnorm as pyln
import sys
import glob
#src = sys.argv[1]
rnn = "/content/rnnoise/examples/rnnoise_demo"

#paths = Path(src).glob("**/*.wav")
#paths = Path(orig_wavs).glob("**/*.wav")
paths = glob.glob(os.path.join(orig_wavs, '*.wav'))
#print(paths)
#paths = Path(orig_wavs).glob("*.wav")

for filepath in paths:
  target_filepath=Path(str(filepath).replace("original", "converted"))
  target_dir=os.path.dirname(target_filepath)
  #print(target_filepath)
  if (str(filepath) == str(target_filepath)):
    raise ValueError("Source and target path are identical: " + str(target_filepath))
  print("From: " + str(filepath))
  print("To: " + str(target_filepath))
	
# Stereo to Mono; upsample to 48000Hz
# added -G to fix gain, -v 0.95
  subprocess.run(["sox", "-G", "-v", "0.95", filepath, "48k.wav", "remix", "-", "rate", "48000"])
  subprocess.run(["sox", "48k.wav", "-c", "1", "-r", "48000", "-b", "16", "-e", "signed-integer", "-t", "raw", "temp.raw"]) # convert wav to raw
  subprocess.run([rnn, "temp.raw", "rnn.raw"]) # apply rnnoise
  subprocess.run(["sox", "-G", "-v", "0.95", "-r", "48k", "-b", "16", "-e", "signed-integer", "rnn.raw", "-t", "wav", "rnn.wav"]) # convert raw back to wav

  subprocess.run(["mkdir", "-p", str(target_dir)])
  subprocess.run(["sox", "rnn.wav", str(target_filepath), "remix", "-", "highpass", "100", "lowpass", "7000", "rate", "22050"]) # apply high/low pass filter and change sr to 22050Hz
  data, rate = sf.read(target_filepath)

# peak normalize audio to -1 dB
  peak_normalized_audio = pyln.normalize.peak(data, -1.0)

# measure the loudness first
  meter = pyln.Meter(rate) # create BS.1770 meter
  loudness = meter.integrated_loudness(data)

# loudness normalize audio to -25 dB LUFS
  loudness_normalized_audio = pyln.normalize.loudness(data, loudness, -25.0)
  sf.write(target_filepath, data=loudness_normalized_audio, samplerate=22050)
  print("")


In [2]:
dataset_name = "new-dataset" #@param {type:"string"}
output_directory = "traineroutput" #@param {type:"string"}



7. Save the training script with your dataset name, and output dir set. File will be saved in your Google drive dataset folder as train_vits.py

In [ ]:
code = """import os

from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

#output_path = os.path.dirname(os.path.abspath(__file__))
##########################################
#Change this to your dataset directory
##########################################
output_path = "/content/drive/MyDrive/"""
code = code + dataset_name + "/" + output_directory + "/" + "\""

code=code + """
dataset_config = BaseDatasetConfig(
##########################################
#Change this to your dataset directory
##########################################
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "/content/drive/MyDrive/"""
code = code + dataset_name
code=code + """")

)
audio_config = VitsAudioConfig(
    sample_rate=22050, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="vits_ljspeech_ly",
    batch_size=8,
    eval_batch_size=8,
    batch_group_size=4,
#    num_loader_workers=8,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100000,
    save_step=1000,
	save_checkpoints=True,
	save_n_checkpoints=3,
	save_best_after=1000,
    text_cleaner="english_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    print_step=50,
    print_eval=True,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    cudnn_benchmark=False,
)

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)
trainer.fit()
"""
#print(code)
myFile = open("/content/drive/MyDrive/" + dataset_name + "/train_vits.py", 'w')

myFile.write(code)
myFile.close()
%ls -al /content/drive/MyDrive/$dataset_name/train_vits.py

-rw------- 1 root root 2946 May  5 10:15 /content/drive/MyDrive/new-dataset/train_vits.py


8. Fine Tune VITS model

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /root/.local/share/tts/tts_models--en--ljspeech--vits/model_file.pth

**Resume a fine tuning session**

A. Run the next cell to list all of the saved run direcories.  Copy select the run you want to resume, ctrl+C to copy the name. Paste in the next cell and run it.

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory

phoneme_cache  vits_ljspeech_ly-May-08-2023_05+58AM-0000000


In [ ]:
run_name = "vits_ljspeech_ly-May-08-2023_05+58AM-0000000" #@param {type:"string"}

B. Run this cell to list all of the checkpoints in the saved run, then put the checkpoint name in the next cell, and run it.

In [ ]:
!ls /content/drive/MyDrive/$dataset_name/$output_directory/$run_name

best_model_1019667.pth	config.json
best_model.pth		events.out.tfevents.1683525511.bfec041a2818
checkpoint_1035000.pth	trainer_0_log.txt
checkpoint_1036000.pth	train_vits.py


In [ ]:
model_checkpoint = "checkpoint_1036000.pth" #@param {type:"string"}

C. Run the next cell to restore a previous fine tuning session using your dataset name, trainer output directory, and model checkpoint set

In [ ]:
!CUDA_VISIBLE_DEVICES="0" python /content/drive/MyDrive/$dataset_name/train_vits.py --restore_path /content/drive/MyDrive/$dataset_name/$output_directory/$run_name/$model_checkpoint --config_path /content/drive/MyDrive/$dataset_name/$run_name/config.json

Streaming output truncated to the last 5000 lines.


> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: espeak
| > Number of instances : 392
 | > Preprocessing samples
 | > Max text length: 145
 | > Min text length: 8
 | > Avg text length: 53.558673469387756
 | 
 | > Max audio length: 176422.0
 | > Min audio length: 17770.0
 | > Avg audio length: 67238.94897959183
 | > Num. instances discarded samples: 0
 | > Batch group size: 32.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessa

View Memory Usage

In [ ]:
!nvidia-smi

Wed May 10 06:03:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Load Tensorboard

In [ ]:
import torch 
%load_ext tensorboard

Load Dashboard. May take several minutes to appear from a blank white box.  Ad blockers probably need to whitelist a bunch of Colab stuff or this won't wrok.

In [ ]:
%tensorboard --logdir /content/drive/MyDrive/$dataset_name/$output_directory/